In [21]:
from ipynb.fs.full.cs_entities import ProjectForAnalysis, SystemBoundaries, AuthorColor
from ipynb.fs.full.cs_analysis import \
    HotspotAnalysis, \
    StaticFileComplexityAnalysis, TrendFileComplexityAnalysis, \
    SystemCouplingAnalysis, FileCouplingAnalysis, \
    ParallelWorkAnalysis, MainDeveloperAnalysis

## Create projects for analysis objects

Need to specify the location of the local git directory, and optionally include a *before* and *after* date (in the form YYYY-MM-DD) to limit the timespan for the analysis.

In [22]:
boundaries_dict = {
    "Code": ["src/code_maat"],
    "Test": ["test/code_maat"],
}
code_maat = ProjectForAnalysis(
    "/home/brombaut/work/code-maat",
    system_boundaries=SystemBoundaries(boundaries_dict)
)

In [23]:
code_maat = ProjectForAnalysis("/home/brombaut/work/code-maat")
sklearn = ProjectForAnalysis("/home/brombaut/work/scikit-learn")

# benrombautca = ProjectForAnalysis("/home/brombaut/work/benrombautca")
# hibernate = ProjectForAnalysis("/home/brombaut/work/hibernate-orm", before="2013-09-05", after="2012-01-01")
# craft = ProjectForAnalysis("/home/brombaut/work/Craft.Net", before="2013-01-01")

### Perform a Hotspot analysis

In [24]:
hs_analysis = HotspotAnalysis(code_maat).analyze()

See what the primary hotspots in the project for the timespan is

In [25]:
hs_analysis.df().head(10)

,module,revisions,code
0,src/code_maat/analysis/logical_coupling.clj,26,145
1,src/code_maat/app/app.clj,25,85
2,src/code_maat/core.clj,21,35
3,test/code_maat/end_to_end/scenario_tests.clj,20,91
4,project.clj,19,17
5,src/code_maat/parsers/svn.clj,19,53
6,src/code_maat/parsers/git.clj,14,31
7,src/code_maat/analysis/authors.clj,14,47
8,test/code_maat/analysis/logical_coupling_test.clj,13,89
9,README.md,11,87


In [44]:
hs_analysis.enclosure_diagram()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### File Complexity Analysis
Based off of file indentation patterns

In [ ]:
static_file_complexity = StaticFileComplexityAnalysis(
    craft, "Craft.Net.Server/MinecraftServer.cs"
)
static_file_complexity.analyze()

##### Current file complexity stats

In [ ]:
static_file_complexity.df()

##### File complexity trends

In [ ]:
file_complexity_trend = TrendFileComplexityAnalysis(
    craft,
    "Craft.Net.Server/MinecraftServer.cs"
)
file_complexity_trend.analyze()
file_complexity_trend.total_trend_line_plot().sd_trend_line_plot()

### Sum of Coupling Analysis

In [ ]:
# soc_analysis = SumOfCouplingAnalysis(craft).analyze()
# soc_analysis.soc_data().head(10)
# ftc_analysis = FileTemporalCouplingAnalysis(craft, "Craft.Net.Server/MinecraftServer.cs").analyze()
# ftc_analysis.enclosure_diagram()

### System Coupling Analysis

In [ ]:
boundaries_dict = {
    "Code": ["src/code_maat"],
    "Analysis Test": ["test/code_maat/analysis"],
    "Dataset Test": ["test/code_maat/dataset"],
    "End to end Test": ["test/code_maat/end_to_end"],
    "Parsers Test": ["test/code_maat/parsers"],
}
boundaries = SystemBoundaries(boundaries_dict)
craft = ProjectForAnalysis("/home/brombaut/work/code-maat", system_boundaries=boundaries)
coupling_data = SystemCouplingAnalysis(craft).analyze().df()
coupling_data

In [ ]:
# Different example
benrombautca = ProjectForAnalysis("/home/brombaut/work/benrombautca")
coupling_data = SystemCouplingAnalysis(benrombautca).analyze().df()
coupling_data.head()

### File Coupling Analysis

In [ ]:
benrombautca = ProjectForAnalysis("/home/brombaut/work/benrombautca")
file_coupling_analysis = FileCouplingAnalysis(benrombautca, "src/bookshelf/BookshelfSection.vue")
file_coupling_analysis.analyze().df()

In [ ]:
file_coupling_analysis.analyze().enclosure_diagram()

### Parallel Work Analysis

In [ ]:
craft = ProjectForAnalysis("/home/brombaut/work/Craft.Net")
an = ParallelWorkAnalysis(craft)
an.analyze()
an.df().head()

In [ ]:
an.enclosure_diagram()

In [ ]:
craft = ProjectForAnalysis("/home/brombaut/work/Craft.Net")
an = MainDeveloperAnalysis(craft)
an.analyze()
an.df().head()

In [ ]:
an.enclosure_diagram()